In [1]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
from pyspark.sql import Row

In [2]:
a=[Row(measurements=[Row(Observation_ID='5', Observation_Name='ABC', Observation_Result='108/72'),
                   Row(Observation_ID='11', Observation_Name='ABC', Observation_Result='70'),
                   Row(Observation_ID='10', Observation_Name='ABC', Observation_Result='73.029'),
                   Row(Observation_ID='14', Observation_Name='XYZ', Observation_Result='23.1')]),
 Row(measurements=[Row(Observation_ID='2', Observation_Name='ZZZ', Observation_Result='3/4'),
                   Row(Observation_ID='5', Observation_Name='ABC', Observation_Result='7')])]

df=spark.createDataFrame(a)

In [3]:
df.show(truncate=False)
df.printSchema()

+---------------------------------------------------------------------+
measurements |
+---------------------------------------------------------------------+
[[5, ABC, 108/72], [11, ABC, 70], [10, ABC, 73.029], [14, XYZ, 23.1]]|
[[2, ZZZ, 3/4], [5, ABC, 7]] |
+---------------------------------------------------------------------+

root
-- measurements: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- Observation_ID: string (nullable = true)
 | |-- Observation_Name: string (nullable = true)
 | |-- Observation_Result: string (nullable = true)

In [4]:
df.withColumn("map", F.create_map(F.lit('Murtihash'),F.array(F.lit(1),F.lit(2)))).drop("measurements")\
  .withColumn("zip", F.explode(F.arrays_zip(F.map_keys("map").alias("keys"),F.map_values("map").alias("values"))))\
  .select("zip.*").select(F.col("".show(truncate=False)

+---------+------+
0 |1 |
+---------+------+
Murtihash|[1, 2]|
Murtihash|[1, 2]|
+---------+------+

In [5]:
df.withColumn("map", F.create_map(F.lit('Murtihash'),F.array(F.lit(1),F.lit(2)))).drop("measurements")\
  .select(F.explode("map").alias("name","score")).show()

+---------+------+
 name| score|
+---------+------+
Murtihash|[1, 2]|
Murtihash|[1, 2]|
+---------+------+

In [6]:
from pyspark.sql import functions as F
df.withColumn("measurements",F.expr("""transform(measurements\
,x-> struct(x.Observation_ID as Observation_ID,\
             x.Observation_Result as Observation_Result))""")).take(2)

Out[42]: [Row(measurements=[Row(Observation_ID='5', Observation_Result='108/72'), Row(Observation_ID='11', Observation_Result='70'), Row(Observation_ID='10', Observation_Result='73.029'), Row(Observation_ID='14', Observation_Result='23.1')]),
 Row(measurements=[Row(Observation_ID='2', Observation_Result='3/4'), Row(Observation_ID='5', Observation_Result='7')])]

In [7]:
df.withColumn("measurements", \
F.expr("""transform(measurements,x-> struct(string(x.Observation_ID) as Observation_ID,string(x.Observation_Result)\
as Observation_Result))""")).printSchema()

root
-- measurements: array (nullable = true)
 |-- element: struct (containsNull = false)
 | |-- Observation_ID: string (nullable = true)
 | |-- Observation_Result: string (nullable = true)

In [8]:
x=[1,2,3]
y=[12,21,31]

print([zip(x,y)])

[<zip object at 0x7f421afd3388>]

In [9]:
datasetDomainList=['SourceSystemName',
 'Portfolio',
 'LastChangedForDelta',
 'IsDeleted',
 'SourceLeaseID',
 'SourcePropertyID',
 'LeaseType',
 'LeaseStatus',
 'AbstractPreparedBy',
 'AbstractReviewedBy',
 'AccountType']

   
  



df=spark.createDataFrame([(i, x) for i, x in enumerate(datasetDomainList)],['index','col1'])
df.show()

+-----+-------------------+
index| col1|
+-----+-------------------+
 0| SourceSystemName|
 1| Portfolio|
 2|LastChangedForDelta|
 3| IsDeleted|
 4| SourceLeaseID|
 5| SourcePropertyID|
 6| LeaseType|
 7| LeaseStatus|
 8| AbstractPreparedBy|
 9| AbstractReviewedBy|
 10| AccountType|
+-----+-------------------+

In [10]:
[(i, x) for i, x in enumerate(arr)]

Out[17]: [(0, 'SourceSystemName'),
 (1, 'Portfolio'),
 (2, 'LastChangedForDelta'),
 (3, 'IsDeleted'),
 (4, 'SourceLeaseID'),
 (5, 'SourcePropertyID'),
 (6, 'LeaseType'),
 (7, 'LeaseStatus'),
 (8, 'AbstractPreparedBy'),
 (9, 'AbstractReviewedBy'),
 (10, 'AccountType')]

In [11]:
list=[['10292016']]

df_imigration=spark.createDataFrame(list,['allowed_date'])

df.show()

+------------+
allowed_date|
+------------+
 10292016|
+------------+

In [12]:
df_imigration.withColumn('cc', F.to_timestamp("allowed_date","MMddyyyy")).show()

+------------+-------------------+
allowed_date| cc|
+------------+-------------------+
 10292016|2016-10-29 00:00:00|
+------------+-------------------+

In [13]:
list=[[[5,2]],
      [[2,5]],
     [[5,5]]]

df=spark.createDataFrame(list,['list'])

df.show()

+------+
 list|
+------+
[5, 2]|
[2, 5]|
[5, 5]|
+------+

In [14]:
df.withColumn("pp",F.expr("""transform(array(list),x -> x[0] - x[1])""")[0]).show()

+------+---+
 list| pp|
+------+---+
[5, 2]| 3|
[2, 5]| -3|
[5, 5]| 0|
+------+---+

In [15]:
df.withColumn("diff", F.expr("""aggregate(list,list[0],(x,acc)-> IF(x!=acc, acc-x,x))""")).show()

+------+----+
 list|diff|
+------+----+
[5, 2]| -3|
[2, 5]| 3|
+------+----+

In [16]:
list=[[['-5594162570~222222-PHONE~FAX-17-TEST','-2812597115~1111111-PHONE']],
       [['-5594162570~222222-PHONE~FAX-17-TEST','-2812597115~1111111-PHONE~FAX-17-TESTB']], 
       [['-2812597115~1111111-PHONE~FAX-17-TESTB','-2812597115~1111111-FAX']],
       [['-2812597115~1111111-PHONE']]]

df=spark.createDataFrame(list,['typed_phone_numbers'])

df.show(truncate=False)

+------------------------------------------------------------------------------+
typed_phone_numbers |
+------------------------------------------------------------------------------+
[-5594162570~222222-PHONE~FAX-17-TEST, -2812597115~1111111-PHONE] |
[-5594162570~222222-PHONE~FAX-17-TEST, -2812597115~1111111-PHONE~FAX-17-TESTB]|
[-2812597115~1111111-PHONE~FAX-17-TESTB, -2812597115~1111111-FAX] |
[-2812597115~1111111-PHONE] |
+------------------------------------------------------------------------------+

In [17]:
#+------------------------------------------------------------------------------+
#|typed_phone_numbers                                                           |
#+------------------------------------------------------------------------------+
#|[-5594162570~222222-PHONE~FAX-17-TEST, -2812597115~1111111-PHONE]             |
#|[-5594162570~222222-PHONE~FAX-17-TEST, -2812597115~1111111-PHONE~FAX-17-TESTB]|
#|[-2812597115~1111111-PHONE~FAX-17-TESTB, -2812597115~1111111-FAX]             |
#|[-2812597115~1111111-PHONE]                                                   |
#+------------------------------------------------------------------------------+


In [18]:
df\
  .withColumn("yo", F.expr("""(transform(typed_phone_numbers,x-> split(substring(x,2,length(x)),'\-|~')))"""))\
  .withColumn("typed_phone_numbers",\
                          F.flatten(F.expr("""transform(yo,y->\
                          IF((array_contains(y,'PHONE')==True) and (array_contains(y,'FAX')==True),\
                                                   array(concat('-',concat_ws('-',y[0],y[2],y[4],y[5])),\
                                                         concat('-',concat_ws('-',y[1],y[3],y[4],y[5]))),\
                                                         array(concat('-',concat_ws('-',y)))))""")))\
                          .drop("yo").show(truncate=False)

+---------------------------------------------------------------------------------------------------+
typed_phone_numbers |
+---------------------------------------------------------------------------------------------------+
[-5594162570-PHONE-17-TEST, -222222-FAX-17-TEST, -2812597115-1111111-PHONE] |
[-5594162570-PHONE-17-TEST, -222222-FAX-17-TEST, -2812597115-PHONE-17-TESTB, -1111111-FAX-17-TESTB]|
[-2812597115-PHONE-17-TESTB, -1111111-FAX-17-TESTB, -2812597115-1111111-FAX] |
[-2812597115-1111111-PHONE] |
+---------------------------------------------------------------------------------------------------+

In [19]:
#+---------------------------------------------------------------------------------------------------+
#|typed_phone_numbers                                                                                |
#+---------------------------------------------------------------------------------------------------+
#|[-5594162570-PHONE-17-TEST, -222222-FAX-17-TEST, -2812597115-1111111-PHONE]                        |
#|[-5594162570-PHONE-17-TEST, -222222-FAX-17-TEST, -2812597115-PHONE-17-TESTB, -1111111-FAX-17-TESTB]|
#|[-2812597115-PHONE-17-TESTB, -1111111-FAX-17-TESTB, -2812597115-1111111-FAX]                       |
#|[-2812597115-1111111-PHONE]                                                                        |
#+---------------------------------------------------------------------------------------------------+

In [20]:
df\
  .withColumn("yo", F.expr("""(transform(typed_phone_numbers,x-> split(substring(x,2,length(x)),'\-|~')))"""))\
  .withColumn("typed_phone_numbers",F.when(F.size(F.expr("""filter(yo[0],x->x='PHONE' or x='FAX')"""))==2,\
                          F.flatten(F.expr("""transform(yo,y->\
                                                   array(concat('-',concat_ws('-',y[0],y[2],y[4],y[5])),\
                                                         concat('-',concat_ws('-',y[1],y[3],y[4],y[5]))))""")))\
                          .otherwise(F.col("typed_phone_numbers")))\
                          .drop("yo").show(truncate=False)

+---------------------------------------------------------------------------------------------------+
typed_phone_numbers |
+---------------------------------------------------------------------------------------------------+
[-5594162570-PHONE-17-TEST, -222222-FAX-17-TEST] |
[-5594162570-PHONE-17-TEST, -222222-FAX-17-TEST, -2812597115-PHONE-17-TESTB, -1111111-FAX-17-TESTB]|
[-2812597115-PHONE-17-TESTB, -1111111-FAX-17-TESTB] |
[-2812597115~1111111-PHONE] |
+---------------------------------------------------------------------------------------------------+

In [21]:
#+---------------------------------------------------------------------------------------------------+
#|typed_phone_numbers                                                                                |
#+---------------------------------------------------------------------------------------------------+
#|[-5594162570-PHONE-17-TEST, -222222-FAX-17-TEST]                                                   |
#|[-5594162570-PHONE-17-TEST, -222222-FAX-17-TEST, -2812597115-PHONE-17-TESTB, -1111111-FAX-17-TESTB]|
#|[-2812597115-PHONE-17-TESTB, -1111111-FAX-17-TESTB]                                                |
#|[-2812597115~1111111-PHONE]                                                                        |
#+---------------------------------------------------------------------------------------------------+

In [22]:
df.withColumn("yo", F.col("typed_phone_numbers")[0])\
  .withColumn("yo", F.split("yo", '\-|~'))\
  .withColumn("yo", F.expr("""filter(yo,x-> x!='')"""))\
  .withColumn("yo", F.array(F.concat(F.lit('-'),F.concat_ws('-',F.element_at("yo",1),\
                                                   F.element_at("yo",3),\
                                                   F.element_at("yo",6),\
                                                   F.element_at("yo",7))),\
                           F.concat(F.lit('-'),F.concat_ws('-',F.element_at("yo",2),\
                                                   F.element_at("yo",5),\
                                                   F.element_at("yo",6),\
                                                   F.element_at("yo",7))))).show(truncate=False)

+----------------------------------------+--------------------------------------------+
typed_phone_numbers |yo |
+----------------------------------------+--------------------------------------------+
[-5594162570~222222-PHONE~FAX-17-TEST] |[-5594162570-PHONE-TEST, -222222-17-TEST] |
[-2812597115~1111111-PHONE~FAX-17-TESTB]|[-2812597115-PHONE-TESTB, -1111111-17-TESTB]|
[-2812597115~1111111-PHONE] |[-2812597115-PHONE, -1111111] |
+----------------------------------------+--------------------------------------------+

In [23]:
from pyspark.sql import functions as F
df.withColumn("yo", F.split(F.col("typed_phone_numbers")[0], '\-|~'))\
  .withColumn("yo", F.expr("""filter(yo,x-> x!='')"""))\
  .withColumn("typed_phone_numbers", F.when(F.size(F.expr("""filter(yo,x-> x='PHONE' or x='FAX')"""))==2,\
                           F.array(F.concat(F.lit('-'),F.concat_ws('-',F.element_at("yo",1),\
                                                   F.element_at("yo",3),\
                                                   F.element_at("yo",5),\
                                                   F.element_at("yo",6))),\
                           F.concat(F.lit('-'),F.concat_ws('-',F.element_at("yo",2),\
                                                   F.element_at("yo",4),\
                                                   F.element_at("yo",5),\
                                                   F.element_at("yo",6)))))\
              .otherwise(F.col("typed_phone_numbers"))).drop("yo").show(truncate=False)

+---------------------------------------------------+
typed_phone_numbers |
+---------------------------------------------------+
[-5594162570-PHONE-17-TEST, -222222-FAX-17-TEST] |
[-2812597115-PHONE-17-TESTB, -1111111-FAX-17-TESTB]|
[-2812597115~1111111-PHONE] |
+---------------------------------------------------+

In [24]:
df.withColumn("yo", F.split(F.col("typed_phone_numbers")[0], '\-|~'))\
  .withColumn("yo", F.array_remove("yo", '')).show()

+--------------------+--------------------+
 typed_phone_numbers| yo|
+--------------------+--------------------+
[-5594162570~2222...|[5594162570, 2222...|
[-2812597115~1111...|[2812597115, 1111...|
[-2812597115~1111...|[2812597115, 1111...|
+--------------------+--------------------+

In [25]:
df.withColumn("yo", F.split(F.col("typed_phone_numbers")[0], '\-|~'))\
  .withColumn("yo", F.array_sort(F.expr("""filter(yo,x-> x!='')"""))).show(truncate=False)

+----------------------------------------+--------------------------------------------+
typed_phone_numbers |yo |
+----------------------------------------+--------------------------------------------+
[-5594162570~222222-PHONE~FAX-17-TEST] |[17, 222222, 5594162570, FAX, PHONE, TEST] |
[-2812597115~1111111-PHONE~FAX-17-TESTB]|[1111111, 17, 2812597115, FAX, PHONE, TESTB]|
[-2812597115~1111111-PHONE] |[1111111, 2812597115, PHONE] |
+----------------------------------------+--------------------------------------------+

In [26]:
df.withColumn("yo", F.split(F.col("typed_phone_numbers")[0], '\-|~'))\
  .withColumn("yo", F.array_remove("yo", ''))\
  .withColumn("typed_phone_numbers", F.when(F.size(F.expr("""filter(yo,x-> x='PHONE' or x='FAX')"""))==2,\
                           F.array(F.concat(F.lit('-'),F.concat_ws('-',F.element_at("yo",1),\
                                                   F.element_at("yo",3),\
                                                   F.element_at("yo",5),\
                                                   F.element_at("yo",6))),\
                           F.concat(F.lit('-'),F.concat_ws('-',F.element_at("yo",2),\
                                                   F.element_at("yo",4),\
                                                   F.element_at("yo",5),\
                                                   F.element_at("yo",6)))))\
              .otherwise(F.col("typed_phone_numbers"))).drop("yo").show(truncate=False)

+---------------------------------------------------+
typed_phone_numbers |
+---------------------------------------------------+
[-5594162570-PHONE-17-TEST, -222222-FAX-17-TEST] |
[-2812597115-PHONE-17-TESTB, -1111111-FAX-17-TESTB]|
[-2812597115~1111111-PHONE] |
+---------------------------------------------------+

In [27]:
from pyspark.sql import functions as F
df.withColumn("yo", F.split(F.col("typed_phone_numbers")[0], '\-|~'))\
  .withColumn("yo", F.expr("""filter(yo,x-> x!='')"""))\
  .withColumn("typed_phone_numbers", F.when(F.size(F.expr("""filter(yo,x-> x='PHONE' or x='FAX')"""))==2,\
                           F.array(F.concat(F.lit('-'),F.concat_ws('-',F.element_at("yo",1),\
                                                   F.element_at("yo",3),\
                                                   F.element_at("yo",5),\
                                                   F.element_at("yo",6))),\
                           F.concat(F.lit('-'),F.concat_ws('-',F.element_at("yo",2),\
                                                   F.element_at("yo",4),\
                                                   F.element_at("yo",5),\
                                                   F.element_at("yo",6)))))\
              .otherwise(F.col("typed_phone_numbers"))).drop("yo").show(truncate=False)

+---------------------------------------------------+
typed_phone_numbers |
+---------------------------------------------------+
[-5594162570-PHONE-17-TEST, -222222-FAX-17-TEST] |
[-2812597115-PHONE-17-TESTB, -1111111-FAX-17-TESTB]|
[-2812597115~1111111-PHONE] |
+---------------------------------------------------+

In [28]:
+-------------------------------------------------------+
|typed_phone_numbers                                    |
+-------------------------------------------------------+
|["-5594162570-PHONE-17-TEST","-222222-FAX-17-TEST"]    |
|["-2812597115-PHONE-17-TESTB","-1111111-FAX-17-TESTB"] |
+-------------------------------------------------------+

In [29]:
df.withColumn("yo", F.col("typed_phone_numbers")[0])\
  .withColumn("yo", F.split("yo", '\-|~'))\
  .withColumn("yo", F.expr("""filter(yo,x-> x!='')"""))\
  .withColumn("typed_phone_numbers", F.when(F.size(F.expr("""filter(yo,x-> x='PHONE' or x='FAX')"""))==2,\
                           F.array(F.concat(F.lit('-'),F.concat_ws('-',F.element_at("yo",1),\
                                                   F.element_at("yo",3),\
                                                   F.element_at("yo",6),\
                                                   F.element_at("yo",6),\
                                                   F.element_at("yo",7))),\
                           F.concat(F.lit('-'),F.concat_ws('-',F.element_at("yo",2),\
                                                   F.element_at("yo",4),\
                                                   F.element_at("yo",6),\
                                                   F.element_at("yo",6),\
                                                   F.element_at("yo",7)))))\
              .otherwise(F.col("typed_phone_numbers"))).drop("yo").show(truncate=False)

+---------------------------------------------------------+
typed_phone_numbers |
+---------------------------------------------------------+
[-5594162570-PHONE-TEST-TEST, -222222-FAX-TEST-TEST] |
[-2812597115-PHONE-TESTB-TESTB, -1111111-FAX-TESTB-TESTB]|
[-2812597115~1111111-PHONE] |
+---------------------------------------------------------+

In [30]:
+-------------------------------------------------------+
|typed_phone_numbers                                    |
+-------------------------------------------------------+
|["-5594162570-PHONE-17-TEST","-222222-FAX-17-TEST"]    |
|["-2812597115-PHONE-17-TESTB","-1111111-FAX-17-TESTB"] |
+-------------------------------------------------------+

In [31]:
+-------------------------------------------------------+
|typed_phone_numbers                                    |
+-------------------------------------------------------+
|["-5594162570-PHONE-17-TEST","-222222-FAX-17-TEST"]    |
|["-2812597115-PHONE-17-TESTB","-1111111-FAX-17-TESTB"] |
+-------------------------------------------------------+

In [32]:
list=[['2019-12-29 00:01:00',9.5],
      ['2019-12-29 00:02:00',9.0],
      ['2019-12-29 00:04:00',8.0],
      ['2019-12-29 00:06:00',21.0],
      ['2019-12-29 00:08:00',7.0],
      ['2019-12-29 00:11:00',8.5],
      ['2019-12-29 00:12:00',11.5],
      ['2019-12-29 00:14:00',8.0],
      ['2019-12-29 00:16:00',31.0],
      ['2019-12-29 00:18:00',8.0],
      ['2019-12-29 00:21:00',8.0],
      ['2019-12-29 00:22:00',16.5],
      ['2019-12-29 00:24:00',7.0],
      ['2019-12-29 00:26:00',14.0],
      ['2019-12-29 00:28:00',7.0]]

df=spark.createDataFrame(list,['event_time','avg_value'])

df.show()

+-------------------+---------+
 event_time|avg_value|
+-------------------+---------+
2019-12-29 00:01:00| 9.5|
2019-12-29 00:02:00| 9.0|
2019-12-29 00:04:00| 8.0|
2019-12-29 00:06:00| 21.0|
2019-12-29 00:08:00| 7.0|
2019-12-29 00:11:00| 8.5|
2019-12-29 00:12:00| 11.5|
2019-12-29 00:14:00| 8.0|
2019-12-29 00:16:00| 31.0|
2019-12-29 00:18:00| 8.0|
2019-12-29 00:21:00| 8.0|
2019-12-29 00:22:00| 16.5|
2019-12-29 00:24:00| 7.0|
2019-12-29 00:26:00| 14.0|
2019-12-29 00:28:00| 7.0|
+-------------------+---------+

In [33]:
def ud(a,b=2):
  return a/b

udf=F.udf(ud, DoubleType())
df.withColumn("avg_value2", udf("avg_value")).show()

+-------------------+---------+----------+
 event_time|avg_value|avg_value2|
+-------------------+---------+----------+
2019-12-29 00:01:00| 9.5| 4.75|
2019-12-29 00:02:00| 9.0| 4.5|
2019-12-29 00:04:00| 8.0| 4.0|
2019-12-29 00:06:00| 21.0| 10.5|
2019-12-29 00:08:00| 7.0| 3.5|
2019-12-29 00:11:00| 8.5| 4.25|
2019-12-29 00:12:00| 11.5| 5.75|
2019-12-29 00:14:00| 8.0| 4.0|
2019-12-29 00:16:00| 31.0| 15.5|
2019-12-29 00:18:00| 8.0| 4.0|
2019-12-29 00:21:00| 8.0| 4.0|
2019-12-29 00:22:00| 16.5| 8.25|
2019-12-29 00:24:00| 7.0| 3.5|
2019-12-29 00:26:00| 14.0| 7.0|
2019-12-29 00:28:00| 7.0| 3.5|
+-------------------+---------+----------+

In [34]:
w=Window().partitionBy().orderBy(F.col("event_time").cast("long")).rangeBetween(Window.currentRow,60*10)

df.withColumn("event_time", F.to_timestamp("event_time"))\
  .withColumn("max", F.max("avg_value").over(w)).show()

+-------------------+---------+----+
 event_time|avg_value| max|
+-------------------+---------+----+
2019-12-29 00:01:00| 9.5|21.0|
2019-12-29 00:02:00| 9.0|21.0|
2019-12-29 00:04:00| 8.0|21.0|
2019-12-29 00:06:00| 21.0|31.0|
2019-12-29 00:08:00| 7.0|31.0|
2019-12-29 00:11:00| 8.5|31.0|
2019-12-29 00:12:00| 11.5|31.0|
2019-12-29 00:14:00| 8.0|31.0|
2019-12-29 00:16:00| 31.0|31.0|
2019-12-29 00:18:00| 8.0|16.5|
2019-12-29 00:21:00| 8.0|16.5|
2019-12-29 00:22:00| 16.5|16.5|
2019-12-29 00:24:00| 7.0|14.0|
2019-12-29 00:26:00| 14.0|14.0|
2019-12-29 00:28:00| 7.0| 7.0|
+-------------------+---------+----+

In [35]:
df.groupBy(F.window("event_time","10 minutes"))\
  .agg(F.element_at(F.array_sort(F.collect_list("event_time")),-2).alias("event_time"),\
       F.max("avg_value").alias("avg_value")).drop("window").orderBy("event_time").show(truncate=False)

+-------------------+---------+
event_time |avg_value|
+-------------------+---------+
2019-12-29 00:06:00|21.0 |
2019-12-29 00:16:00|31.0 |
2019-12-29 00:26:00|16.5 |
+-------------------+---------+

In [36]:
df.groupBy(F.window("event_time","10 minutes"))\
  .agg(F.collect_list(F.struct("event_time","avg_value")).alias("event_time")\
       ,F.max("avg_value").alias("avg_value"))\
  .withColumn("event_time", F.expr("""filter(event_time, x-> x.avg_value=avg_value)"""))\
        .select((F.col("event_time.event_time")[0]).alias("event_time"),"avg_value").orderBy("event_time").show()

+-------------------+---------+
 event_time|avg_value|
+-------------------+---------+
2019-12-29 00:06:00| 21.0|
2019-12-29 00:16:00| 31.0|
2019-12-29 00:22:00| 16.5|
+-------------------+---------+

In [37]:
df.groupBy(F.window("event_time","10 minutes"))\
  .agg(F.collect_list(F.struct("event_time","avg_value")).alias("event_time")\
       ,F.max("avg_value").alias("avg_value"))\
  .withColumn("event_time", F.expr("""filter(event_time, x-> x.avg_value=avg_value)"""))\
        .select((F.col("event_time.event_time")[0]).alias("event_time"),"avg_value").orderBy("event_time").show()

+-------------------+---------+
 event_time|avg_value|
+-------------------+---------+
2019-12-29 00:06:00| 21.0|
2019-12-29 00:16:00| 31.0|
2019-12-29 00:22:00| 16.5|
+-------------------+---------+

In [38]:
df.withColumn("yo", F.explode(F.array(F.struct('event_time'),F.struct(F.lit('avg_value')))))\
  .select("yo.*").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o283.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve 'array(named_struct('event_time', `event_time`), named_struct('col1', 'avg_value'))' due to data type mismatch: input to function array should all be the same type, but it's [struct<event_time:string>, struct<col1:string>];;
'Project [event_time#16, avg_value#17, explode(array(named_struct(event_time, event_time#16), named_struct(col1, avg_value))) AS yo#1660]
+- LogicalRDD [event_time#16, avg_value#17], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.immutabl

In [39]:
+-------------------+---------+
|         event_time|avg_value|
+-------------------+---------+
|2019-12-29 00:06:00|     21.0|
|2019-12-29 00:16:00|     31.0|
|2019-12-29 00:22:00|     16.5|
+-------------------+---------+eve

In [40]:
from pyspark.sql.functions import window, max, col

w = Window().partitionBy('group_col')

(
    df.
        withColumn(
            'group_col',
            window('event_time', '10 minutes')
        ).
        withColumn(
            'max_val',
            max(col('avg_value')).over(w)
        ).
        where(
            col('avg_value') == col('max_val')
        ).
        drop(
            'max_val',
            'group_col'
        ).
        orderBy('event_time').
        show(truncate=False)
)

+-------------------+---------+
event_time |avg_value|
+-------------------+---------+
2019-12-29 00:06:00|21.0 |
2019-12-29 00:16:00|31.0 |
2019-12-29 00:22:00|16.5 |
+-------------------+---------+